**Libraries Required**

In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from nltk import sent_tokenize
import nltk
from numpy import linalg as lin
import unicodedata
import pickle
from sklearn.cluster import KMeans

In [2]:
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

In [3]:
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [4]:
def cosine_similarity(a,b):
    try:
        return a.dot(b)/(lin.norm(a)*lin.norm(b))
    except:
        return -1.0

In [5]:
def ed(a,b):
    return lin.norm(a-b)

In [6]:
data=pd.read_csv('../English.csv')

In [7]:
file=open('sentence.pkl','rb')
verse_embeddings_sentence=pickle.load(file)
file.close()
file=open('max.pkl','rb')
verse_embeddings_max=pickle.load(file)
file.close()
file=open('mean.pkl','rb')
verse_embeddings_mean=pickle.load(file)
file.close()

In [8]:
file=open('whole.pkl','rb')
verse_embeddings_whole=pickle.load(file)
file.close()

In [12]:
query='I want to die'
query_embedding=model.encode(query)
query_embedding_list=query_embedding.reshape(1,-1).astype(np.float64)

# Max Strategy

## Cosine Similarity

In [10]:
def max_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [11]:
verse_index=max_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
589,18,72,कच्च‍िदेतच्छ्रुतं पार्थ त्वयैकाग्रेण चेतसा ।कच...,The Lord was acting as the spiritual master of...
587,18,68,य इदं परमं गुह्यं मद्भ‍क्तेष्वभिधास्यति ।भक्ति...,Generally it is advised that Bhagavad-gītā be ...
595,18,78,यत्र योगेश्वर: कृष्णो यत्र पार्थो धनुर्धर: ।तत...,The Bhagavad-gītā began with an inquiry of Dhṛ...
182,4,41,योगसन्न्यस्तकर्माणं ज्ञानसञ्छिन्नसंशयम् ।आत्मव...,One who follows the instruction of the Bhagava...
585,18,66,सर्वधर्मान्परित्यज्य मामेकं शरणं व्रज ।अहं त्व...,The Lord has described various kinds of knowle...
72,2,45,त्रैगुण्यविषया वेदा निस्त्रैगुण्यो भवार्जुन ।न...,All material activities involve actions and re...
492,15,20,इति गुह्यतमं शास्त्रमिदमुक्तं मयानघ ।एतद्‍बुद्...,The Lord clearly explains here that this is th...
143,4,2,एवं परम्पराप्राप्तमिमं राजर्षयो विदुः ।स कालेन...,It is clearly stated that the Gītā was especia...
40,2,11,श्री भगवानुवाचअशोच्यनन्वशोचस्त्वं प्रज्ञावादां...,The Lord at once took the position of the teac...
337,9,34,मन्मना भव मद्भ‍क्तो मद्याजी मां नमस्कुरु ।मामे...,In this verse it is clearly indicated that Kṛṣ...


## Euclidean Distance

In [12]:
def max_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [13]:
verse_index=max_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
587,18,68,य इदं परमं गुह्यं मद्भ‍क्तेष्वभिधास्यति ।भक्ति...,Generally it is advised that Bhagavad-gītā be ...
589,18,72,कच्च‍िदेतच्छ्रुतं पार्थ त्वयैकाग्रेण चेतसा ।कच...,The Lord was acting as the spiritual master of...
595,18,78,यत्र योगेश्वर: कृष्णो यत्र पार्थो धनुर्धर: ।तत...,The Bhagavad-gītā began with an inquiry of Dhṛ...
182,4,41,योगसन्न्यस्तकर्माणं ज्ञानसञ्छिन्नसंशयम् ।आत्मव...,One who follows the instruction of the Bhagava...
143,4,2,एवं परम्पराप्राप्तमिमं राजर्षयो विदुः ।स कालेन...,It is clearly stated that the Gītā was especia...
585,18,66,सर्वधर्मान्परित्यज्य मामेकं शरणं व्रज ।अहं त्व...,The Lord has described various kinds of knowle...
492,15,20,इति गुह्यतमं शास्त्रमिदमुक्तं मयानघ ।एतद्‍बुद्...,The Lord clearly explains here that this is th...
40,2,11,श्री भगवानुवाचअशोच्यनन्वशोचस्त्वं प्रज्ञावादां...,The Lord at once took the position of the teac...
144,4,3,स एवायं मया तेऽद्य योगः प्रोक्तः पुरातनः ।भक्त...,"There are two classes of men, namely the devot..."
72,2,45,त्रैगुण्यविषया वेदा निस्त्रैगुण्यो भवार्जुन ।न...,All material activities involve actions and re...


# Mean Pooling

## Cosine Similarity

In [13]:
def mean_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [14]:
verse_index=mean_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
63,2,34,अकीर्तिं चापि भूतानि कथयिष्यन्ति तेऽव्ययाम् ।स...,"Both as friend and philosopher to Arjuna, Lord..."
224,6,25,शनै: शनैरुपरमेद्‍बुद्ध्या धृतिगृहीतया ।आत्मसंस...,By proper conviction and intelligence one shou...
282,8,6,यं यं वापि स्मरन्भावं त्यजत्यन्ते कलेवरम् ।तं ...,The process of changing one’s nature at the cr...
278,8,2,अधियज्ञ: कथं कोऽत्र देहेऽस्मिन्मधुसूदन ।प्रयाण...,“Lord of sacrifice” may refer to either Indra ...
336,9,33,किं पुनर्ब्राह्मणा: पुण्या भक्ता राजर्षयस्तथा ...,In this material world there are classificatio...
466,14,15,रजसि प्रलयं गत्वा कर्मसङ्गिषु जायते ।तथा प्रली...,Some people have the impression that when the ...
19,1,30,न च शक्न‍ोम्यवस्थातुं भ्रमतीव च मे मनः ।निमित्...,"Due to his impatience, Arjuna was unable to st..."
467,14,16,कर्मण: सुकृतस्याहु: सात्त्विकं निर्मलं फलम् ।र...,The result of pious activities in the mode of ...
76,2,49,दूरेण ह्यवरं कर्म बुद्धियोगाद्धनञ्जय ।बुद्धौ श...,One who has actually come to understand one’s ...
500,16,16,अनेकचित्तविभ्रान्ता मोहजालसमावृता: ।प्रसक्ता: ...,The demoniac man knows no limit to his desire ...


## Euclidean Distance

In [16]:
data.loc[63]['Commentary']

'Both as friend and philosopher to Arjuna, Lord Kṛṣṇa now gives His final judgment regarding Arjuna’s refusal to fight. The Lord says, “Arjuna, if you leave the battlefield before the battle even begins, people will call you a coward. And if you think that people may call you bad names but that you will save your life by fleeing the battlefield, then My advice is that you’d do better to die in the battle. For a respectable man like you, ill fame is worse than death. So, you should not flee for fear of your life; better to die in the battle. That will save you from the ill fame of misusing My friendship and from losing your prestige in society.”So, the final judgment of the Lord was for Arjuna to die in the battle and not withdraw.'

In [16]:
def mean_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [17]:
verse_index=mean_ed(query_embedding)

In [18]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
143,4,2,एवं परम्पराप्राप्तमिमं राजर्षयो विदुः ।स कालेन...,It is clearly stated that the Gītā was especia...
587,18,68,य इदं परमं गुह्यं मद्भ‍क्तेष्वभिधास्यति ।भक्ति...,Generally it is advised that Bhagavad-gītā be ...
589,18,72,कच्च‍िदेतच्छ्रुतं पार्थ त्वयैकाग्रेण चेतसा ।कच...,The Lord was acting as the spiritual master of...
142,4,1,श्रीभगवानुवाचइमं विवस्वते योगं प्रोक्तवानहमव्य...,Herein we find the history of the Bhagavad-gīt...
530,18,1,सन्न्यासस्य महाबाहो तत्त्वमिच्छामि वेदितुम् ।त...,Actually the Bhagavad-gītā is finished in seve...
0,1,1,धृतराष्ट्र उवाचधर्मक्षेत्रे कुरुक्षेत्रे समवेत...,Bhagavad-gītā is the widely read theistic scie...
183,4,42,तस्मादज्ञानसम्भूतं हृत्स्थं ज्ञानासिनात्मनः ।छ...,The yoga system instructed in this chapter is ...
487,15,15,सर्वस्य चाहं हृदि सन्निविष्टोमत्त: स्मृतिर्ज्ञ...,The Supreme Lord is situated as Paramātmā in e...
303,8,28,वेदेषु यज्ञेषु तप:सु चैवदानेषु यत्पुण्यफलं प्र...,This verse is the summation of the Seventh and...
72,2,45,त्रैगुण्यविषया वेदा निस्त्रैगुण्यो भवार्जुन ।न...,All material activities involve actions and re...


# Line Matching

## Cosine Similarity

In [19]:
def line_cosine(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[cosine_similarity(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [20]:
verse_index=line_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
128,3,30,मयि सर्वाणि कर्माणि सन्न्यस्याध्यात्मचेतसा ।नि...,This verse clearly indicates the purpose of th...
586,18,67,इदं ते नातपस्काय नाभक्ताय कदाचन ।न चाश‍ुश्रूषव...,Persons who have not undergone the austerities...
413,11,55,मत्कर्मकृन्मत्परमो मद्भ‍क्त: सङ्गवर्जित: ।निर्...,Anyone who wants to approach the supreme of al...
261,7,15,न मां दुष्कृतिनो मूढा: प्रपद्यन्ते नराधमा: ।मा...,It is said in Bhagavad-gītā that simply by sur...
491,15,19,यो मामेवमसम्मूढो जानाति पुरुषोत्तमम् ।स सर्ववि...,There are many philosophical speculations abou...
143,4,2,एवं परम्पराप्राप्तमिमं राजर्षयो विदुः ।स कालेन...,It is clearly stated that the Gītā was especia...
270,7,24,अव्यक्तं व्यक्तिमापन्नं मन्यन्ते मामबुद्धय: ।प...,Those who are worshipers of demigods have been...
435,13,19,इति क्षेत्रं तथा ज्ञानं ज्ञेयं चोक्तं समासत: ।...,"The Lord has described in summary the body, kn..."
41,2,12,न त्वेवाहं जातु नासं न त्वं नेमे जनाधिपाः ।न च...,In the Vedas – in the Kaṭha Upaniṣad as well a...
45,2,16,नासतो विद्यते भावो नाभावो विद्यते सतः ।उभयोरपि...,There is no endurance of the changing body. Th...


## Euclidean Distance

In [21]:
def line_ed(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[ed(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [22]:
verse_index=line_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
567,18,40,न तदस्ति पृथिव्यां वा दिवि देवेषु वा पुन: ।सत्...,The Lord here summarizes the total influence o...
66,2,37,हतो वा प्राप्स्यसि स्वर्ग जित्वा वा भोक्ष्यसे ...,Even though there was no certainty of victory ...
531,18,2,काम्यानां कर्मणां न्यासं सन्न्यासं कवयो विदु: ...,The performance of activities for results has ...
179,4,38,न हि ज्ञानेन सदृशं पवित्रमिह विद्यते ।तत्स्वयं...,"When we speak of transcendental knowledge, we ..."
248,7,2,ज्ञानं तेऽहं सविज्ञानमिदं वक्ष्याम्यशेषत: ।यज्...,Complete knowledge includes knowledge of the p...
591,18,74,इत्यहं वासुदेवस्य पार्थस्य च महात्मन: ।संवादमि...,"In the beginning of Bhagavad-gītā, Dhṛtarāṣṭra..."
557,18,29,बुद्धेर्भेदं धृतेश्चैव गुणतस्त्रिविधं श‍ृणु ।प...,"Now after explaining knowledge, the object of ..."
528,17,25,तदित्यनभिसन्धाय फलं यज्ञतप:क्रिया: ।दानक्रियाश...,"To be elevated to the spiritual position, one ..."
517,17,13,विधिहीनमसृष्टान्नं मन्‍त्रहीनमदक्षिणम् ।श्रद्ध...,Faith in the mode of darkness or ignorance is ...
391,11,21,अमी हि त्वां सुरसङ्घा विशन्तिकेचिद्भ‍ीता: प्रा...,The demigods in all the planetary systems fear...


# Whole Part

## Cosine Similarity

In [23]:
def whole_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [24]:
verse_index=whole_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
305,9,2,राजविद्या राजगुह्यं पवित्रमिदमुत्तमम् ।प्रत्यक...,This chapter of Bhagavad-gītā is called the ki...
435,13,19,इति क्षेत्रं तथा ज्ञानं ज्ञेयं चोक्तं समासत: ।...,"The Lord has described in summary the body, kn..."
142,4,1,श्रीभगवानुवाचइमं विवस्वते योगं प्रोक्तवानहमव्य...,Herein we find the history of the Bhagavad-gīt...
0,1,1,धृतराष्ट्र उवाचधर्मक्षेत्रे कुरुक्षेत्रे समवेत...,Bhagavad-gītā is the widely read theistic scie...
143,4,2,एवं परम्पराप्राप्तमिमं राजर्षयो विदुः ।स कालेन...,It is clearly stated that the Gītā was especia...
593,18,76,राजन्संस्मृत्य संस्मृत्य संवादमिममद्भुतम् ।केश...,The understanding of Bhagavad-gītā is so trans...
128,3,30,मयि सर्वाणि कर्माणि सन्न्यस्याध्यात्मचेतसा ।नि...,This verse clearly indicates the purpose of th...
583,18,64,सर्वगुह्यतमं भूय: श‍ृणु मे परमं वच: ।इष्टोऽसि ...,The Lord has given Arjuna knowledge that is co...
181,4,40,अज्ञश्चाश्रद्दधानश्च संशयात्मा विनश्यति ।नायं ...,Out of many standard and authoritative reveale...
491,15,19,यो मामेवमसम्मूढो जानाति पुरुषोत्तमम् ।स सर्ववि...,There are many philosophical speculations abou...


## Euclidean Distance

In [25]:
def whole_ed(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [26]:
verse_index=whole_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
186,5,3,ज्ञेय: स नित्यसन्न्यासी यो न द्वेष्टि न काङ्‍क...,One who is fully in Kṛṣṇa consciousness is alw...
411,11,53,नाहं वेदैर्न तपसा न दानेन न चेज्यया ।शक्य एवंव...,Kṛṣṇa first appeared before His parents Devakī...
28,1,45,यदि मामप्रतीकारमशस्त्रं शस्त्रपाणयः ।धार्तराष्...,It is the custom – according to kṣatriya fight...
522,17,19,मूढग्राहेणात्मनो यत्पीडया क्रियते तप: ।परस्योत...,There are instances of foolish penance underta...
83,2,56,दुःखेष्वनुद्विग्न‍मनाः सुखेषु विगतस्पृहः ।वीतर...,The word muni means one who can agitate his mi...
52,2,23,नैनं छिन्दन्ति शस्त्राणि नैनं दहति पावकः ।न चै...,"All kinds of weapons – swords, flame weapons, ..."
194,5,13,सर्वकर्माणि मनसा सन्न्यस्यास्ते सुखं वशी ।नवद्...,The embodied soul lives in the city of nine ga...
594,18,77,तच्च‍ संस्मृत्य संस्मृत्य रूपमत्यद्भुतं हरे: ।...,"It appears that Sañjaya also, by the grace of ..."
495,16,6,द्वौ भूतसर्गौ लोकेऽस्मिन्दैव आसुर एव च ।दैवो व...,"Lord Kṛṣṇa, having assured Arjuna that he was ..."
481,15,9,श्रोत्रं चक्षु: स्पर्शनं च रसनं घ्राणमेव च ।अध...,"In other words, if the living entity adulterat..."


# Clustering

## Max Embedding

In [27]:
kmeans_max = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_max)

In [28]:
def kmeans_max_cluster(query_embedding_list):
    label=kmeans_max.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_max.labels_[i]==label]
    return verse_index

In [29]:
verse_index=kmeans_max_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
72,2,45,त्रैगुण्यविषया वेदा निस्त्रैगुण्यो भवार्जुन ।न...,All material activities involve actions and re...
93,2,67,इन्द्रियाणां हि चरतां यन्मनोऽनुविधीयते ।तदस्य ...,Unless all of the senses are engaged in the se...
98,2,72,एषा ब्राह्मी स्थितिःपार्थ नैनां प्राप्य विमुह्...,One can attain Kṛṣṇa consciousness or divine l...
104,3,6,कर्मेन्द्रियाणि संयम्य य आस्ते मनसा स्मरन् ।इन...,There are many pretenders who refuse to work i...
182,4,41,योगसन्न्यस्तकर्माणं ज्ञानसञ्छिन्नसंशयम् ।आत्मव...,One who follows the instruction of the Bhagava...
221,6,18,यदा विनियतं चित्तमात्मन्येवावतिष्ठते ।निस्पृह:...,The activities of the yogī are distinguished f...
337,9,34,मन्मना भव मद्भ‍क्तो मद्याजी मां नमस्कुरु ।मामे...,In this verse it is clearly indicated that Kṛṣ...
495,16,6,द्वौ भूतसर्गौ लोकेऽस्मिन्दैव आसुर एव च ।दैवो व...,"Lord Kṛṣṇa, having assured Arjuna that he was ..."
579,18,60,स्वभावजेन कौन्तेय निबद्ध: स्वेन कर्मणा ।कर्तुं...,If one refuses to act under the direction of t...
587,18,68,य इदं परमं गुह्यं मद्भ‍क्तेष्वभिधास्यति ।भक्ति...,Generally it is advised that Bhagavad-gītā be ...


## Mean Embedding

In [30]:
kmeans_mean = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_mean)

In [31]:
def kmeans_mean_cluster(query_embedding_list):
    label=kmeans_mean.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_mean.labels_[i]==label]
    return verse_index

In [32]:
verse_index=kmeans_mean_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
39,2,10,तमुवाच हृषीकेशः प्रहसन्निव भारत ।सेनयोरूभयोर्म...,The talk was going on between intimate friends...
73,2,46,यावानर्थ उदपाने सर्वतः सम्प्लुतोदके ।तावान्सर्...,The rituals and sacrifices mentioned in the ka...
100,3,2,व्यामिश्रेणेव वाक्येन बुद्धिं मोहयसीव मे ।तदेक...,"In the previous chapter, as a prelude to the B..."
142,4,1,श्रीभगवानुवाचइमं विवस्वते योगं प्रोक्तवानहमव्य...,Herein we find the history of the Bhagavad-gīt...
143,4,2,एवं परम्पराप्राप्तमिमं राजर्षयो विदुः ।स कालेन...,It is clearly stated that the Gītā was especia...
144,4,3,स एवायं मया तेऽद्य योगः प्रोक्तः पुरातनः ।भक्त...,"There are two classes of men, namely the devot..."
248,7,2,ज्ञानं तेऽहं सविज्ञानमिदं वक्ष्याम्यशेषत: ।यज्...,Complete knowledge includes knowledge of the p...
303,8,28,वेदेषु यज्ञेषु तप:सु चैवदानेषु यत्पुण्यफलं प्र...,This verse is the summation of the Seventh and...
304,9,1,श्रीभगवानुवाचइदं तु ते गुह्यतमं प्रवक्ष्याम्यन...,As a devotee hears more and more about the Sup...
323,9,20,त्रैविद्या मां सोमपा: पूतपापायज्ञैरिष्ट्वा स्व...,The word trai-vidyāḥ refers to the three Vedas...


## Whole Embedding

In [33]:
kmeans_whole = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_whole)

In [34]:
def kmeans_whole_cluster(query_embedding_list):
    label=kmeans_whole.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_whole.labels_[i]==label]
    return verse_index

In [35]:
verse_index=kmeans_whole_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
36,2,7,कार्पण्यदोषोपहतस्वभावःपृच्छामि त्वां धर्मसम्मू...,By nature’s own way the complete system of mat...
104,3,6,कर्मेन्द्रियाणि संयम्य य आस्ते मनसा स्मरन् ।इन...,There are many pretenders who refuse to work i...
113,3,15,कर्म ब्रह्मोद्भ‍वं विद्धि ब्रह्माक्षरसमुद्भ‍वम...,"Yajñārtha-karma, or the necessity of work for ..."
128,3,30,मयि सर्वाणि कर्माणि सन्न्यस्याध्यात्मचेतसा ।नि...,This verse clearly indicates the purpose of th...
175,4,34,तद्विद्धि प्रणिपातेन परिप्रश्न‍ेन सेवया ।उपदेक...,The path of spiritual realization is undoubted...
183,4,42,तस्मादज्ञानसम्भूतं हृत्स्थं ज्ञानासिनात्मनः ।छ...,The yoga system instructed in this chapter is ...
216,6,8,ज्ञानविज्ञानतृप्‍तात्मा कूटस्थो विजितेन्द्रिय:...,Book knowledge without realization of the Supr...
240,6,41,प्राप्य पुण्यकृतां लोकानुषित्वा शाश्वती: समा: ...,The unsuccessful yogīs are divided into two cl...
303,8,28,वेदेषु यज्ञेषु तप:सु चैवदानेषु यत्पुण्यफलं प्र...,This verse is the summation of the Seventh and...
304,9,1,श्रीभगवानुवाचइदं तु ते गुह्यतमं प्रवक्ष्याम्यन...,As a devotee hears more and more about the Sup...
